In [1]:
import os
import json
import time
import torch
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers.util import cos_sim
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer
from sklearn.metrics.pairwise import cosine_similarity

/home/un/anaconda3/envs/GLM4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
train_data = pd.read_json("/home/un/桌面/QC/2024_全国大数据智能大赛/new_复赛_code/data/dev.json")
rule_data=pd.read_json("/home/un/桌面/QC/2024_全国大数据智能大赛/new_复赛_code/data/rules1.json")

In [4]:
train_data.head()

,question_id,question_text,answer,rule_id
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463]
1,2,问题：在国家重大海上溢油应急处置工作中，确保有序、高效的应对是至关重要的。根据规则，应急队伍...,C,[390]
2,3,问题：在某次海上搜救任务中，搜救中心收到了一条紧急信息，一艘载有200人的客轮在远洋航行中遇...,C,[167]
3,4,问题：在国家海洋局领导面对日益严峻的风暴潮、海浪、海啸和海冰灾害形势下，决定采取更为有效的应...,C,[467]
4,5,问题：在2032年10月15日，一架国际航班的民用航空器在飞往上海的途中，在上海市郊外发生了...,A,"[202, 217]"


In [5]:
rule_data.head()

,rule_id,rule_text
0,1,危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化...
1,2,协调指挥机构与职责：在国务院及国务院安委会统一领导下，安全监管总局负责统一指导、协调危险化学...
2,3,办公厅的职责：负责应急值守，及时向安全监管总局领导报告事故信息，传达安全监管总局领导关于事故...
3,4,政策法规司的职责：负责事故信息发布工作，与中宣部、国务院新闻办及新华社、人民日报社、中央人民...
4,5,安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参...


In [5]:
# rule_data.loc[[int(i)-1 for i in train_data["rule_id"][0]]]["rule_text"].values[0]

In [6]:

# # 创建一个空列用于存放规则文本
# train_data['rule_data'] = None

# # 遍历每一条记录，并填充rule_data列
# for index, row in train_data.iterrows():
#     rule_ids = row['rule_id']
#     rule_texts = []
#     for rule_id in rule_ids:
#         rule_texts.append(rule_data.loc[int(rule_id) - 1, 'rule_text'])
#     train_data.at[index, 'rule_data'] = rule_texts

In [7]:
# train_data.head()

In [6]:
# model = SentenceTransformer(r'/home/un/桌面/QC/雨季同学/Langchain-Chatchat-master/bge-large-zh-v1.5', trust_remote_code=True)
# model = SentenceTransformer(r'/home/tom/fsas/model/iic/gte_Qwen2-1___5B-instruct', trust_remote_code=True)

model = SentenceTransformer(r'/home/un/桌面/QC/2024_全国大数据智能大赛/rag/MiniCPM-Embedding', trust_remote_code=True)
model = model.to(torch.bfloat16)



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.37it/s]


In [7]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MiniCPMModel 
  (1): Pooling({'word_embedding_dimension': 2304, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [8]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    TaskType.FEATURE_EXTRACTION,
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

In [9]:
model = get_peft_model(model, config)

In [10]:
def embedding_s(data):
    l=[]
    for i in data:
        l.append(model.encode(i))
    return l

In [11]:
positive_data=embedding_s(train_data["question_text"])
anchor_data=embedding_s(rule_data["rule_text"])

In [12]:
import numpy as np
train_cos_sim_arr = cosine_similarity(positive_data, anchor_data)
train_sorted_indices = np.argsort(-train_cos_sim_arr, axis=1)

In [13]:
train_sorted_indices=train_sorted_indices+1

In [14]:
train_sorted_indices

array([[493, 485, 463, ..., 621, 599, 600],
       [390, 369, 337, ..., 676, 672, 673],
       [167, 168, 176, ..., 621,  94,  91],
       ...,
       [451, 452, 453, ...,  67, 744,   5],
       [295,  46, 242, ..., 599, 666, 670],
       [ 29,  32,  34, ..., 600, 670, 599]])

In [15]:
top_indices = train_sorted_indices[:, :10]

# 转换成列表格式
top_indices_list = [list(row) for row in top_indices]

# 添加到 DataFrame
train_data['predictrule_id'] = top_indices_list

In [16]:
train_data.head()

,question_id,question_text,answer,rule_id,predictrule_id
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],"[493, 485, 463, 477, 470, 486, 478, 494, 487, ..."
1,2,问题：在国家重大海上溢油应急处置工作中，确保有序、高效的应对是至关重要的。根据规则，应急队伍...,C,[390],"[390, 369, 337, 360, 338, 335, 358, 339, 392, ..."
2,3,问题：在某次海上搜救任务中，搜救中心收到了一条紧急信息，一艘载有200人的客轮在远洋航行中遇...,C,[167],"[167, 168, 176, 170, 172, 173, 165, 181, 174, ..."
3,4,问题：在国家海洋局领导面对日益严峻的风暴潮、海浪、海啸和海冰灾害形势下，决定采取更为有效的应...,C,[467],"[467, 465, 464, 459, 482, 474, 432, 490, 472, ..."
4,5,问题：在2032年10月15日，一架国际航班的民用航空器在飞往上海的途中，在上海市郊外发生了...,A,"[202, 217]","[217, 203, 201, 202, 210, 238, 226, 211, 222, ..."


In [17]:
train_exploded = train_data.explode('predictrule_id')

In [18]:
train_exploded

,question_id,question_text,answer,rule_id,predictrule_id
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],493
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],485
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],463
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],477
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],470
...,...,...,...,...,...
499,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],30
499,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],225
499,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],20
499,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],35


In [19]:
predict_mapping = rule_data.add_prefix('Predict').rename(columns={'Predictrule_id': 'predictrule_id'})

In [20]:
predict_mapping["rule_id"]=list(range(len(predict_mapping)))

In [21]:
predict_mapping["rule_id"]=predict_mapping["rule_id"]+1

In [22]:
predict_mapping.head()

,predictrule_id,Predictrule_text,rule_id
0,1,危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化...,1
1,2,协调指挥机构与职责：在国务院及国务院安委会统一领导下，安全监管总局负责统一指导、协调危险化学...,2
2,3,办公厅的职责：负责应急值守，及时向安全监管总局领导报告事故信息，传达安全监管总局领导关于事故...,3
3,4,政策法规司的职责：负责事故信息发布工作，与中宣部、国务院新闻办及新华社、人民日报社、中央人民...,4
4,5,安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参...,5


In [23]:
train_joined_once = train_exploded.merge(predict_mapping, left_on='predictrule_id', right_on='predictrule_id', how='left')

In [24]:
train_joined_once.head()

,question_id,question_text,answer,rule_id_x,predictrule_id,Predictrule_text,rule_id_y
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],493,Ⅳ级应急响应应急加密观测：海冰灾害影响期间，北海分局组织相关中心站和海洋站每周开展1次重点岸...,493
1,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],485,Ⅲ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,485
2,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],463,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463
3,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],477,Ⅱ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,477
4,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],470,Ⅰ级应急响应数据传输与分发：预报中心、海区预报中心、中心站、海洋站各级数据传输节点应加强数据...,470


In [27]:
# # 基于 MisconceptionId 进行第一次 join
# train_joined_once = train_exploded.merge(predict_mapping, on='rule_id', how='left')

In [25]:
# 创建一个空列用于存放规则文本
train_joined_once['rule_data'] = None

# 遍历每一条记录，并填充rule_data列
for index, row in train_joined_once.iterrows():
    rule_ids = row['rule_id_x']
    rule_texts = []
    for rule_id in rule_ids:
        rule_texts.append(rule_data.loc[int(rule_id) - 1, 'rule_text'])
    train_joined_once.at[index, 'rule_data'] = "\n".join(rule_texts)

In [26]:
train_joined_once["label_id"]=[int(i[0]) for i in train_joined_once["rule_id_x"]]

In [27]:
train_joined_once

,question_id,question_text,answer,rule_id_x,predictrule_id,Predictrule_text,rule_id_y,rule_data,label_id
0,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],493,Ⅳ级应急响应应急加密观测：海冰灾害影响期间，北海分局组织相关中心站和海洋站每周开展1次重点岸...,493,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463
1,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],485,Ⅲ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,485,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463
2,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],463,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463
3,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],477,Ⅱ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,477,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463
4,1,问题：在应对海冰灾害过程中，北海分局组织相关中心站和海洋站对受影响的重点岸段进行巡视与观测。...,A,[463],470,Ⅰ级应急响应数据传输与分发：预报中心、海区预报中心、中心站、海洋站各级数据传输节点应加强数据...,470,Ⅰ级应急响应应急加密观测：海浪灾害影响期间，受影响海区的分局组织开展海浪加密观测工作。海浪自...,463
...,...,...,...,...,...,...,...,...,...
4995,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],30,根据危险化学品事故可能造成的后果，将危险化学品事故分为：火灾事故、爆炸事故、易燃、易爆或有毒...,30,针对危险化学品事故的特点，现场一般处置方案如下：（1）接警。接警时应明确发生事故的单位名称、...,29
4996,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],225,群众的安全防护：事故现场应急指挥部负责组织事故发生区域群众的安全防护工作。事故现场应急指挥部...,225,针对危险化学品事故的特点，现场一般处置方案如下：（1）接警。接警时应明确发生事故的单位名称、...,29
4997,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],20,分级响应原则：事故发生后，发生事故的企业及其所在地政府立即启动应急预案，并根据事故等级及时上...,20,针对危险化学品事故的特点，现场一般处置方案如下：（1）接警。接警时应明确发生事故的单位名称、...,29
4998,500,问题：在一次化学品厂发生的氨气泄漏事故中，应急管理部门接到报警后迅速响应。事故现场位于工业区...,A,[29],35,事故分析、检测与后果评估：当地和支援的环境监测及化学品检测机构负责对水源、空气、土壤等样品就...,35,针对危险化学品事故的特点，现场一般处置方案如下：（1）接警。接警时应明确发生事故的单位名称、...,29


In [28]:
NUM_PROC = os.cpu_count()

In [29]:
int(train_joined_once["rule_id_x"][0][0])

463

In [30]:
train = (
    Dataset.from_pandas(train_joined_once)
    .filter(  # To create an anchor, positive, and negative structure, delete rows where the positive and negative are identical.
        lambda example: example["label_id"] != example["predictrule_id"],
        num_proc=NUM_PROC,
    )
)

Filter (num_proc=32): 100%|██████████| 5000/5000 [00:00<00:00, 6129.68 examples/s]


In [31]:
train

Dataset({
    features: ['question_id', 'question_text', 'answer', 'rule_id_x', 'predictrule_id', 'Predictrule_text', 'rule_id_y', 'rule_data', 'label_id'],
    num_rows: 4506
})

In [32]:
# model = SentenceTransformer("/home/un/桌面/QC/2024_全国大数据智能大赛/rag/MiniCPM-Embedding", trust_remote_code=True)

loss = MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="/home/un/桌面/QC/2024_全国大数据智能大赛/new_复赛_code/fineturne_minincpm_embedding/miniCPMv3",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    # per_device_eval_batch_size=BS,
    # eval_accumulation_steps=GRAD_ACC_STEP,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    # fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    #bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    lr_scheduler_type="cosine_with_restarts",
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=2,
    logging_steps=50,
    # report_to=REPORT_TO,  # Will be used in W&B if `wandb` is installed
    # run_name=EXP_NAME,
    do_eval=False
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train.select_columns(
        ["question_text", "rule_data", "Predictrule_text"]
    ),
    loss=loss
)

trainer.train()
# model.save_pretrained("/home/tom/fssd/ckpt/miniCPMv2")

[2024-11-05 21:27:30,123] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/un/anaconda3/envs/GLM4/lib/python3.12/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/home/un/anaconda3/envs/GLM4/lib/python3.12/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd


Step,Training Loss
50,0.297500
100,0.243000
150,0.204800
200,0.230000
250,0.248200
300,0.224800
350,0.203800
400,0.166100
450,0.138300
500,0.154200


TrainOutput(global_step=843, training_loss=0.19465977893888173, metrics={'train_runtime': 1289.195, 'train_samples_per_second': 10.486, 'train_steps_per_second': 0.654, 'total_flos': 0.0, 'train_loss': 0.19465977893888173, 'epoch': 2.9906790945406128})

### 验证指标

In [36]:
#model = SentenceTransformer(r'/home/un/桌面/QC/雨季同学/Langchain-Chatchat-master/bge-large-zh-v1.5', trust_remote_code=True)

In [33]:
def embedding_s(data,model):
    l=[]
    for i in data:
        l.append(model.encode(i))
    return l

In [35]:
import pandas as pd
train_data = pd.read_json("/home/un/桌面/QC/2024_全国大数据智能大赛/data/复赛新增训练参考集.json")
rule_data=pd.read_json("/home/un/桌面/QC/2024_全国大数据智能大赛/new_复赛_code/my_data/all_rules_data.json")

In [36]:
positive_data=embedding_s(train_data["question_text"],model)
anchor_data=embedding_s(rule_data["rule_text"],model)


In [37]:
import numpy as np
train_cos_sim_arr = cosine_similarity(positive_data, anchor_data)
train_sorted_indices = np.argsort(-train_cos_sim_arr, axis=1)

In [38]:
train_sorted_indices=train_sorted_indices+1

In [39]:
top_indices = train_sorted_indices[:, :10]

# 转换成列表格式
top_indices_list = [list(row) for row in top_indices]

# 添加到 DataFrame
train_data['predictrule_id_new'] = top_indices_list

In [40]:
train_data

,question_id,question_text,refered_rules,predictrule_id_new
0,1,问题：在国家面临重大海上溢油灾害时，需要迅速有效地组织应急处置工作以减少环境污染和经济损失。...,NaN,"[337, 338, 378, 339, 336, 340, 368, 362, 361, ..."
1,2,问题：在处理2025年发生在国际海域的一起重大海上溢油事件时，卫生计生委扮演了至关重要的角色...,NaN,"[394, 351, 362, 347, 342, 357, 341, 380, 363, ..."
2,3,问题：设想目前东岭城及其相邻的几个省份正在遭受一场严重的霾污染，中央气象台已发布霾红色预警，...,"[707. 大雾Ⅱ级响应启动：当中央气象台发布大雾红色预警,且预计未来72h预警区内的大部地...","[710, 709, 708, 707, 706, 705, 690, 704, 700, ..."
3,4,问题：在某国，由于频发的矿山事故，引发了广泛的社会关注和政府的重视。为了进一步加强矿山安全监...,NaN,"[298, 245, 299, 297, 265, 266, 254, 277, 241, ..."
4,5,问题：在某省发生严重的暴雨事件，根据规定启动了Ⅲ级气象灾害应急响应。此时局应急办需要迅速采取...,[661. Ⅲ级响应:签署启动或变更到Ⅲ级应急响应命令后，局应急办向有关省（区、市）气象局、...,"[661, 660, 663, 662, 644, 655, 489, 482, 414, ..."
...,...,...,...,...
4995,4996,问题：在一次发生在2023年5月的重大海上溢油事件中，需要紧急调动各种资源进行处置工作。根据...,NaN,"[396, 397, 398, 400, 871, 378, 196, 392, 368, ..."
4996,4997,问题：某城市突然遭遇了一场沙尘暴灾害，这场沙尘暴广泛影响了城市及其周边地区。根据初步统计，此...,[502. 按照突发沙尘暴灾害的严重性和危害程度，将突发沙尘暴灾害分为4级。特大沙尘暴灾害（...,"[502, 503, 505, 504, 523, 524, 525, 537, 539, ..."
4997,4998,问题：在一座人口密集的大城市中，突然爆发了一种肺鼠疫病例，这种疾病是由Yersinia pe...,[551. 突发公共卫生事件的分级:根据突发公共卫生事件性质、危害程度、涉及范围，突发公共卫...,"[551, 303, 302, 301, 578, 569, 304, 315, 548, ..."
4998,4999,问题：在江南省发生了8.0级的地震，地震影响范围广泛，造成严重的人员伤亡和财产损失。根据应急...,NaN,"[86, 50, 841, 87, 83, 51, 712, 13, 84, 49]"


In [41]:
label_rule_data=train_data[~train_data["refered_rules"].isna()]
label_rule_data['rule_id'] = label_rule_data['refered_rules'].apply(lambda x: [int(y.split('.')[0]) for y in x])

/tmp/ipykernel_413821/3513580278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_rule_data['rule_id'] = label_rule_data['refered_rules'].apply(lambda x: [int(y.split('.')[0]) for y in x])


In [42]:
label_rule_data.head()

,question_id,question_text,refered_rules,predictrule_id_new,rule_id
2,3,问题：设想目前东岭城及其相邻的几个省份正在遭受一场严重的霾污染，中央气象台已发布霾红色预警，...,"[707. 大雾Ⅱ级响应启动：当中央气象台发布大雾红色预警,且预计未来72h预警区内的大部地...","[710, 709, 708, 707, 706, 705, 690, 704, 700, ...","[707, 710]"
4,5,问题：在某省发生严重的暴雨事件，根据规定启动了Ⅲ级气象灾害应急响应。此时局应急办需要迅速采取...,[661. Ⅲ级响应:签署启动或变更到Ⅲ级应急响应命令后，局应急办向有关省（区、市）气象局、...,"[661, 660, 663, 662, 644, 655, 489, 482, 414, ...",[661]
10,11,问题：在某地发生了一次大面积停电事件，影响了超过50万居民的生活和众多企业的运行。政府迅速行...,[947. 后期处置的事件调查：大面积停电事件发生后，根据有关规定成立调查组，查明事件原因、...,"[947, 929, 608, 993, 943, 946, 956, 230, 944, ...",[947]
14,15,问题：在某大城市的城市轨道交通运营中，一段重要的地上线路位于多雨地区，气象部门预报未来48小...,[971. 研判可能发生运营突发事件时，运营单位采取以下防范措施：对于城市轨道交通系统内设施...,"[971, 970, 968, 973, 982, 972, 974, 969, 981, ...",[971]
15,16,问题：在某城市的地铁线路上，由于设备故障导致列车延误，运营单位预测这将可能演变成较大的运营突...,[973. 研判可能发生运营突发事件时，运营单位应做好舆论引导：预警信息发布后，及时公布咨询...,"[973, 970, 966, 971, 972, 987, 975, 978, 967, ...",[973]


In [43]:
def hit_at_k(actual, predicted, k):
    """
    计算预测结果中前k个位置包含的实际正例的数量，并将其与实际正例数量的比例作为结果。

    参数:
    actual (list): 实际的正例列表。
    predicted (list): 预测的结果列表。
    k (int): 考虑的前k个位置。

    返回:
    float: 前k个位置中实际正例的命中比例（命中数/实际正例数）。
    """
    # 截取预测列表的前k个元素
    top_k = predicted[:k]
    # 计算命中数
    hits = sum(item in top_k for item in actual)
    # 计算命中比例
    return hits / len(actual) if len(actual) > 0 else 0

def calculate_weighted_hit(actual, predicted):
    """
    计算不同K值下的Hit@K，并返回加权后的Hit得分。

    参数:
    actual (list): 实际的正例列表。
    predicted (list): 预测的结果列表。

    返回:
    dict: 包含各个Hit@K的得分和加权后的Hit得分。
    """
    ks = [3, 5, 7, 10]  # 不同的k值
    hits = {f'Hit@{k}': hit_at_k(actual, predicted, k) for k in ks}
    weights = [3, 2, 1, 1]  # 权重数组

    # 计算加权平均
    weighted_sum = sum(hits[f'Hit@{k}'] * weight for k, weight in zip(ks, weights))
    total_weight = sum(weights)
    weighted_hit_score = weighted_sum / total_weight

    # 将加权后的得分添加到结果字典中
    hits['Weighted Hit'] = weighted_hit_score

    return hits

In [44]:
pred_labels=[]
pred_result=[]
for i, row in label_rule_data.iterrows():
    # print(i)
    actual_positives = [int(i) for i in row["rule_id"]]
    # print(row["question_text"])
    predicted_order = row["predictrule_id_new"]
    # print(actual_positives)
    # print(predicted_order)
    pred_labels.append(predicted_order)
    # 计算各个Hit@K得分及加权后的得分
    results = calculate_weighted_hit(actual_positives, predicted_order)
    pred_result.append(results)
    # # 打印结果
    # for key, value in results.items():
    #     # print(f"{key}: {value}")
    #     pred_map3

    # # 打印加权后的Hit得分
    # print(f"Weighted Hit Score: {results['Weighted Hit']}")

In [45]:
pred_results=pd.DataFrame(pred_result)

In [46]:
pred_results

,Hit@3,Hit@5,Hit@7,Hit@10,Weighted Hit
0,0.5,1.0,1.0,1.0,0.785714
1,1.0,1.0,1.0,1.0,1.000000
2,1.0,1.0,1.0,1.0,1.000000
3,1.0,1.0,1.0,1.0,1.000000
4,1.0,1.0,1.0,1.0,1.000000
...,...,...,...,...,...
2995,1.0,1.0,1.0,1.0,1.000000
2996,1.0,1.0,1.0,1.0,1.000000
2997,1.0,1.0,1.0,1.0,1.000000
2998,1.0,1.0,1.0,1.0,1.000000


In [47]:
pred_results.mean()

Hit@3           0.925167
Hit@5           0.961056
Hit@7           0.975611
Hit@10          0.986500
Weighted Hit    0.951389
dtype: float64

In [50]:
pred_results.mean()

Hit@3           0.943333
Hit@5           0.972111
Hit@7           0.981556
Hit@10          0.988778
Weighted Hit    0.963508
dtype: float64